In [ ]:
from AnimalShelter import AnimalShelter
import pandas as pd
from getpass import getpass  # hides password input


"""
Cell 1: User Authentication Loop

This cell prompts the user for email and password credentials in the
console and attempts to authenticate with the Firebase backend
through the AnimalShelter class. The loop continues until
authentication succeeds or the user opts to exit.

The use of getpass ensures password input is not displayed, 
enhancing security for demonstrations or screenshots.

This cell acts as a gatekeeper: if authentication fails, the 
dashboard will not proceed.
"""

# Initialize shelter object as None before authentication
shelter = None

# Authentication loop: repeat until a valid AnimalShelter instance is created
while not shelter:
    email = input("Enter your email: ")  # Prompt user for email
    password = getpass("Enter your password: ")  # Prompt for password securely

    try:
        # Attempt to authenticate user
        shelter = AnimalShelter(email=email, password=password)
        print(f"Authenticated as: {email}")  # Confirmation on successful login
    except Exception as e:
        # Report failed authentication and allow retry or exit
        print(f"Login failed: {str(e)}")
        retry = input("Try again? (y/n): ").strip().lower()
        if retry != 'y':
            print("Exiting dashboard.")
            break

In [ ]:
# Cell 2: Dashboard Layout and Callbacks
# ---------------------------------------
# Sets up the AnimalShelter dashboard using Dash.
# Loads data from Firestore, prepares it for display, and defines:
# - Dashboard layout with logo, title, rescue type selector, data table, pie chart, and map.
# - Callbacks for filtering data, updating views, and interacting with the table and map.

from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import base64
import pandas as pd
import plotly.express as px

from AnimalShelter import AnimalShelter

###########################
# Data Preparation
###########################
if not shelter:
    raise SystemExit("No valid login. Dashboard cannot run.")

# Load all documents from Firestore into DataFrame
data = shelter.read({})
df = pd.DataFrame(data)

# Remove MongoDB-specific "_id" field if present
if "_id" in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Sort records by record number
df.sort_values(by="rec_num", ascending=True, inplace=True)

#########################
# Layout
#########################
app = Dash(__name__)

# Load and encode logo
image_filename = 'Grazioso Salvare Logo.png'
with open(image_filename, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Rescue type options
rescue_options = [
    {'label': 'Water', 'value': 'Water'},
    {'label': 'Mountain/Wilderness', 'value': 'Mountain'},
    {'label': 'Disaster/Individual Tracking', 'value': "Disaster"},
    {'label': 'Reset', 'value': "Reset"},
]

# Filtering logic based on rescue type
def filter_data(rescue_type):
    if rescue_type == 'Water':
        return df[(df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
                  (df['sex_upon_outcome'].str.contains('Intact Female', na=False)) &
                  (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_type == 'Mountain':
        return df[(df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
                  (df['sex_upon_outcome'].str.contains('Intact Male', na=False)) &
                  (df['age_upon_outcome_in_weeks'].between(26, 156))]
    elif rescue_type == 'Disaster':
        return df[(df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                  (df['sex_upon_outcome'].str.contains('Intact Male', na=False)) &
                  (df['age_upon_outcome_in_weeks'].between(20, 300))]
    return df

# Dashboard layout
app.layout = html.Div([
    # Logo and title
    html.Center(
        html.A(
            href="https://www.snhu.edu",
            target="_blank",
            children=[
                html.Img(
                    src=f"data:image/png;base64,{encoded_image}",
                    style={'width': '200px', 'border': '2px solid black'}
                )
            ]
        )
    ),
    html.Center(
        html.B([
            html.H1('SNHU CS-340 Dashboard modified by Sonny Coutu'),
            html.Br(),
            html.H2('Updated for CS499 Capstone')
        ])
    ),
    html.Hr(),

    # Rescue type selector
    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.RadioItems(id='rescue-filter', options=rescue_options, value='Reset', inline=True)
    ]),
    html.Br(),

    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        filter_action="native",
        sort_action="native",
        style_cell={'textAlign': 'left'}
    ),
    html.Br(),
    html.Hr(),

    # Pie chart and map layout
    html.Div(className='row',
             style={'display': 'flex'}, children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'flex': '1', 'maxWidth': '400px'},
            children=[dcc.Graph(id='pie-chart', style={'height': '400px'})]
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'flex': '2'}
        )
    ])
])

#########################
# Callbacks
#########################

# Filter data and update table & pie chart
@app.callback(
    [Output('datatable-id', 'data'), Output('pie-chart', 'figure')],
    [Input('rescue-filter', 'value')]
)
def update_dashboard(rescue_type):
    filtered_df = filter_data(rescue_type)
    pie_chart = px.pie(filtered_df, names='sex_upon_outcome', title='Distribution by Sex') \
        if not filtered_df.empty else px.pie(title='No Data Available')
    return filtered_df.to_dict('records'), pie_chart

# Select row when a cell is clicked
@app.callback(
    Output('datatable-id', 'selected_rows'),
    Input('datatable-id', 'active_cell'),
    prevent_initial_call=True
)
def update_selected_row(active_cell):
    if active_cell:
        return [active_cell['row']]
    return []

# Highlight the selected row
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if not selected_rows:
        return []
    return [{
        'if': {'row_index': selected_rows[0]},
        'background_color': '#D2F3FF'
    }]

# Update map display based on selected row
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "data"),
        Input('datatable-id', "selected_rows")
    ]
)
def update_map(table_data, selected_rows):
    if not table_data or not selected_rows:
        return [html.P("No data available to display on map")]

    dff = pd.DataFrame(table_data)
    required_cols = ['location_lat', 'location_long']
    if dff.empty or not all(col in dff.columns for col in required_cols):
        return [html.P("Invalid data format, can't display map")]

    row = selected_rows[0]
    if row >= len(dff):
        return [html.P("Select a valid row")]

    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[float(dff.iloc[row]['location_lat']), float(dff.iloc[row]['location_long'])],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[float(dff.iloc[row]['location_lat']), float(dff.iloc[row]['location_long'])],
                    children=[
                        dl.Tooltip(dff.iloc[row].get('breed', "unknown")),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row].get('name', "No name"))
                        ])
                    ]
                )
            ]
        )
    ]

#########################
# Run the App
#########################
if __name__ == '__main__':
    app.run(debug=False, use_reloader=False, port=8050)
